In [1]:
!pip install confluent_kafka sseclient

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 14.1 MB/s eta 0:00:0000:0100:01
  Created wheel for sseclient: filename=sseclient-0.0.27-py3-none-any.whl size=5563 sha256=49b7f649d8644b1ed087308d3b637ff44a4e6e0bfd252aa9dd524f34d101be38
  Stored in directory: /home/jovyan/.cache/pip/wheels/7c/54/eb/a223b1599728ecaf0528281c17c96c503aa7d18a752a4e4e3a
Successfully built sseclient


In [1]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, random

brokers = "kafka1:9092,kafka2:9093"

In [2]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer':  StringSerializer('utf_8')
}

In [3]:
p = SerializingProducer(pconf)

In [4]:
taxies = "sample.csv"

In [5]:
from datetime import datetime  
  #2023-10-13T08:16:13Z
def construct_ride(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ") #"%d-%m-%Y, %H:%M:%S"
    ride = {"vendor_id": row[2],
             "pickup_datetime": row[5],
             "droppff_datetime":row[5],
            "pickup_longitude": float(row[10]),
            "pickup_latitude": float(row[11]),
            "dropoff_longitude": float(row[12]),
            "dropoff_latitude": float(row[13]),
             "timestamp":str_date_time
             }
    return ride

In [7]:
import csv, json
import time
n = 0
with open(taxies) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader, None) # Skipping the headers
    try:
        for row in csv_reader:
            print(row)
            if n == 10:
               break
            ride = construct_ride(row)
            print(ride)
            p.produce('ride', value=json.dumps(ride))
            p.poll(0)
            #p.flush()
            time.sleep(0.5)
            n = n + 1
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))


['89D227B655E5C82AECF13C3F540D4CF4', 'BA96DE419E711691B9445D6A6307C170', 'CMT', '1', 'N', '2013-01-01 15:11:48', '2013-01-01 15:18:10', '4', '382', '1.00', '-73.978165', '40.757977', '-73.989838', '40.751171']
{'vendor_id': 'CMT', 'pickup_datetime': '2013-01-01 15:11:48', 'droppff_datetime': '2013-01-01 15:11:48', 'pickup_longitude': '-73.978165', 'pickup_latitude': '40.757977', 'dropoff_longitude': '-73.989838', 'dropoff_latitude': '40.751171', 'timestamp': '2024-05-07T06:53:59Z'}
['0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', '1', 'N', '2013-01-06 00:18:35', '2013-01-06 00:22:54', '1', '259', '1.50', '-74.006683', '40.731781', '-73.994499', '40.75066']
{'vendor_id': 'CMT', 'pickup_datetime': '2013-01-06 00:18:35', 'droppff_datetime': '2013-01-06 00:18:35', 'pickup_longitude': '-74.006683', 'pickup_latitude': '40.731781', 'dropoff_longitude': '-73.994499', 'dropoff_latitude': '40.75066', 'timestamp': '2024-05-07T06:53:59Z'}
['0BD7C8F5BA12B88E0B67BED28B

### Use only if you need to purge all the messages in the queue

In [ ]:
brokers = "kafka1:9092,kafka2:9093"

admin_client = AdminClient({"bootstrap.servers":brokers})
admin_client.delete_topics(topics=['stock'])